# TaskWeaver

This notebook aims to condence all requirements needed for coding up a TaskWeaver project.  <br>
https://export.arxiv.org/pdf/2311.17541 <br>

#### Step 1:
- Firstly, you need to make a project folder. this is a prerequisite to running TaskWeaver. See [starter_project]("../_starter_projects/") <br>

#### Step 2:
- Then we need to make a session:
    > TaskWeaver supports a basic session management to keep different users' <br>
    > data separate. The code execution is separated into different processes <br>
    > in order not to interfere with each other.

#### Step 3:
- We also need to enable modules that the code interpreter can use and code verification.
    > Code verification - TaskWeaver is designed to verify the generated code before execution. <br>
    > It can detect potential issues in the generated code and provide suggestions to fix them.
    
#### Step 4:
- TaskWeaver has restricted library use to mitigate against unforeseen consequences of running code, thus you need to call out the permitted libraries expicitly.
- Add this to the `taskweaver_config.json`:
    ```
    "code_verification.allowed_modules": ["pandas", "matplotlib", "numpy", "sklearn", "scipy", "seaborn", "datetime", "typing"],
    "code_verification.code_verification_on": true,
    ```

In [5]:
import json
from typing import Any, Optional

import pprint as pp
from taskweaver.app.app import TaskWeaverApp

# Create a session
# would prefer if the expected folder structure was autogenerated
app_dir = "./_starter_projects/"
app = TaskWeaverApp(app_dir=app_dir)
session = app.get_session()

### Configuration

In [6]:
# Here is where the configurations from taskweaver_config.json are stored
# https://github.com/microsoft/TaskWeaver/blob/main/docs/configuration.md 
# Categories of keys within the config
# - session_manager
# - llm
# - workspace
# - logging
# - session
# - planner
# - plugin
# - round_compressor
# - code_generator
# - embedding_model
# - code_verification
# - code_interpreter
session.config.src.config


{'session_manager.store_type': AppConfigItem(name='session_manager.store_type', value='in_memory', type='enum', sources=[AppConfigSourceValue(source='default', value='in_memory')]),
 'llm.api_type': AppConfigItem(name='llm.api_type', value='openai', type='str', sources=[AppConfigSourceValue(source='default', value='openai')]),
 'llm.embedding_api_type': AppConfigItem(name='llm.embedding_api_type', value='openai', type='str', sources=[AppConfigSourceValue(source='default', value='openai')]),
 'llm.api_base': AppConfigItem(name='llm.api_base', value=None, type='str', sources=[AppConfigSourceValue(source='default', value=None)]),
 'llm.api_key': AppConfigItem(name='llm.api_key', value=None, type='str', sources=[AppConfigSourceValue(source='default', value=None)]),
 'llm.model': AppConfigItem(name='llm.model', value=None, type='str', sources=[AppConfigSourceValue(source='default', value=None)]),
 'llm.backup_model': AppConfigItem(name='llm.backup_model', value=None, type='str', sources=[Ap

### Simple test message

In [7]:
# Send a message to the session
user_query = "hello, what can you do?"
response_round = session.send_message(
                            user_query,
                            event_handler=lambda _type, _msg: print(f"{_type}:\n{_msg}")
                          )

init_plan:
1. Provide information about the capabilities and services offered.
plan:
1. Provide information about the capabilities and services offered.
current_plan_step:
Providing information about the capabilities and services offered.
send_to:
User
message:
Hello! I can assist you by breaking down tasks into subtasks and coordinating with CodeInterpreter to execute them. This includes tasks like data analysis, file operations, and generating Python code to accomplish specific objectives. How can I assist you today?
Planner->User:
Hello! I can assist you by breaking down tasks into subtasks and coordinating with CodeInterpreter to execute them. This includes tasks like data analysis, file operations, and generating Python code to accomplish specific objectives. How can I assist you today?
final_reply_message:
Hello! I can assist you by breaking down tasks into subtasks and coordinating with CodeInterpreter to execute them. This includes tasks like data analysis, file operations, and

In [13]:
response_round.to_dict()['post_list'][-1]['message']

'Hello! I can assist you by breaking down tasks into subtasks and coordinating with CodeInterpreter to execute them. This includes tasks like data analysis, file operations, and generating Python code to accomplish specific objectives. How can I assist you today?'

In [8]:
# Heres what that looks like in JSON
pp.pprint(response_round.to_dict())

{'id': 'round-20231214-223036-7696666b',
 'post_list': [{'attachment_list': [],
                'id': 'post-20231214-223036-a4519d69',
                'message': 'hello, what can you do?',
                'send_from': 'User',
                'send_to': 'Planner'},
               {'attachment_list': [{'content': '1. Provide information about '
                                                'the capabilities and services '
                                                'offered.',
                                     'id': 'atta-20231214-223046-7154b8b1',
                                     'type': 'init_plan'},
                                    {'content': '1. Provide information about '
                                                'the capabilities and services '
                                                'offered.',
                                     'id': 'atta-20231214-223046-a1f1944c',
                                     'type': 'plan'},
                            

### The classic stock market agent benchmark

protip: make sure `llm.response_format` is `text` because when working with `json` i found it crashed when using the code interpreter.

> 💡 Up to 11/30/2023, the json_object and text options of `llm.response_format` is only supported by the OpenAI models later than `1106`. If you are using an older version of OpenAI model, you need to set the llm.response_format to null.

In [15]:
user_query = "Write code to plot the price of tesla stock YTD and save it to the current working directory"
response_round = session.send_message(user_query,
                                      event_handler=lambda _type, _msg: print(f"{_type}:\n{_msg}"))

init_plan:
1. Fetch Tesla stock price data YTD
2. Plot the stock price data
3. Save the plot to the current working directory
plan:
1. Fetch Tesla stock price data YTD
2. Plot the stock price data
3. Save the plot to the current working directory
current_plan_step:
Fetching Tesla stock price data YTD
send_to:
CodeInterpreter
message:
Please fetch the year-to-date (YTD) stock price data for Tesla (TSLA) and store it in a DataFrame.
Planner->CodeInterpreter:
Please fetch the year-to-date (YTD) stock price data for Tesla (TSLA) and store it in a DataFrame.
CodeInterpreter->CodeInterpreter:
Your previous message is not following the output format. You must generate the output as a JSON object with the following format:
{"response": [{"type":"this is the type", "content": "this is the content"}, ...]}
You need at least have an element with type 'python' and content being the code to be executed.
Don't surround the JSON with ```json and ```, just send the JSON object directly.
Please try aga

![](./_starter_projects/tesla_stock_price_ytd.png)

![](../_starter_projects/workspace/sessions/20231212-200217-2ba37e82/cwd/tesla_stock_price_ytd.png)

### Plugins

A plugin is the equivalent to a function call in Autogen

In [ ]:
import os
import pandas as pd
from injector import Injector

from taskweaver.config.config_mgt import AppConfigSource
from taskweaver.logging import LoggingModule
from taskweaver.memory import Memory, Post, Round
from taskweaver.memory.plugin import PluginModule, PluginRegistry
from taskweaver.planner import Planner
from taskweaver.plugin import Plugin, register_plugin, test_plugin

# Plugin definition
@register_plugin
class DataAnalysisPlugin(Plugin):
    def __call__(self, df: pd.DataFrame):
        """
        Performs a simple analysis on the DataFrame and returns a description.
        """
        row_count = len(df)
        column_count = len(df.columns)
        description = f"The DataFrame has {row_count} rows and {column_count} columns."
        return df, description

In [ ]:
# Setup TaskWeaver environment
app_injector = Injector([PluginModule, LoggingModule])
app_config = AppConfigSource(config={
    "plugin.base_path": os.path.join(app_dir, "plugins"),
    "llm.api_key": "******",
})

# app.
app_injector.binder.bind(AppConfigSource, to=app_config)

plugin_registry = app_injector.get(PluginRegistry)
planner = app_injector.create_object(Planner)

# Create a memory object for the session
memory = Memory(session_id="session-1")

def send_message_to_session(query):
    # Create a new round with the user query
    round1 = Round.create(user_query=query, id="round-1")
    post1 = Post.create(message=query, send_from="User", send_to="Planner", attachment_list=[])
    round1.add_post(post1)

    # Add the round to the memory and compose the prompt
    memory.conversation.add_round(round1)
    prompt = planner.compose_prompt(rounds=memory.conversation.rounds)

    # Here you should process the prompt using the appropriate plugin or logic
    # For demonstration, I'm just printing the prompt
    print("Processing:", prompt)

    # Process the result and print the response (This is where your plugin logic will go)
    # Assuming a response is generated
    response = "Processed response for: " + query
    print("Response:\n", response)

    # Update the conversation with the response
    response_post = Post.create(message=response, send_from="Planner", send_to="User", attachment_list=[])
    round1.add_post(response_post)

    return round1

# Example usage
user_query = "Tell me about the data"
round  = send_message_to_session(user_query)